#### Finetune.ipynb  
By using scatter plot dataset from Generate Dataset.ipynb,   
Fine-tune the pre-trained DEPLOT model from huggingface.  


## Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

project_dir = "/content/drive/My Drive/Colab Notebooks/COSE474 Deep Learning HW/20242R0136COSE47402/Final Project"
image_dir = pkl_dir = project_dir + "/data/scatterplots"
# 이미지 데이터는 image_dir에,

Mounted at /content/drive


In [ ]:
!pip install transformers datasets
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.9 MB/s eta 0:00:00


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
from tqdm import tqdm

from datasets import Dataset, DatasetDict
from PIL import Image
from tqdm import tqdm
import pickle

with open(pkl_dir+'/dataset.pkl', 'rb') as f:
    loaded_dataset = pickle.load(f)

def load_image_and_text(item):
    image = Image.open(item['image'])
    text = item['text']
    return image, text

# 이미지 로드 (비교적 빠름)
with ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(load_image_and_text, loaded_dataset), total=len(loaded_dataset)))

tied_dataset = {
    'image': [result[0] for result in results],
    'text': [result[1] for result in results]
}

dataset = Dataset.from_dict(tied_dataset)


100%|██████████| 10500/10500 [09:40<00:00, 18.09it/s]


In [ ]:
with open(pkl_dir + '/dataset_tied.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
"""
# image와 text로 묶은 dataset_tied.pkl 파일이 이미 있다면 불러오기
import pickle
with open(pkl_dir + '/dataset_tied.pkl', 'rb') as f:
    loaded_tied_dataset = pickle.load(f)
"""

In [ ]:
# shuffle dataset

shuffled_dataset = dataset.shuffle(seed=42) # 데이터셋 섞기. shuffle 전 데이터셋은 | 0-3999 : Random | 4000-9499 : Weak Linear | 9500-10500 : Multiple Groups |

In [ ]:
from datasets import Dataset, DatasetDict

dataset_dict = DatasetDict({
    "train": shuffled_dataset.select(range(2000)),  # 첫 2000개를 train으로
    "validation": shuffled_dataset.select(range(2000, 2500))  # 다음 500개를 validation으로
})
test_dataset = shuffled_dataset.select(range(8000, len(dataset))) #마지막 2500개를 test로
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['image', 'text'],
        num_rows: 500
    })
})

In [ ]:
# 디스크에 중간저장
with open(pkl_dir + '/dataset_shuffled.pkl', 'wb') as f:
    pickle.dump(shuffled_dataset, f)
with open(pkl_dir + '/dataset_test.pkl', 'wb') as f:
    pickle.dump(test_dataset, f)

# DEPLOT Output (No use, test only)

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText
import torch

device = torch.device("cuda")
processor = AutoProcessor.from_pretrained("google/deplot")
model = AutoModelForImageTextToText.from_pretrained("google/deplot").to(device)

KeyboardInterrupt: 

In [ ]:
from PIL import Image
import os

path = "/content/drive/My Drive/Colab Notebooks/COSE474 Deep Learning HW/20242R0136COSE47402/Final Project"

# get images from /data/images directory
filenames =[]
images = []
for filename in os.listdir((path+'/data/images')):
  filenames.append(filename)
  images.append(Image.open(path+'/data/images/'+filename))
print(filenames)
images[2]

In [ ]:
inputs = processor(images=images[2], text="Generate underlying data table of the figure below:", return_tensors="pt").to(device)
predictions = model.generate(**inputs, max_new_tokens=512)
table = processor.decode(predictions[0], skip_special_tokens=True)
print(table.replace('<0x0A>','\n'))
print(predictions)

#Fine-tune DEPLOT

In [ ]:
import torch
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, TrainingArguments, Trainer
#from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, TrainingArguments, Trainer
from datasets import load_dataset
from torch.nn import CrossEntropyLoss
from PIL import Image

import os

# Check for CUDA availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load pre-trained model and processor
processor = Pix2StructProcessor.from_pretrained("google/deplot")
model = Pix2StructForConditionalGeneration.from_pretrained("google/deplot").to(device)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/851k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

### Test with 3 samples (No use, test only)

In [ ]:
# prompt: get dataset.pkl with pickle

from datasets import Dataset, DatasetDict
import pickle
from PIL import Image

with open('dataset.pkl', 'rb') as f:
    loaded_dataset = pickle.load(f)

loaded_dataset[0]["image"]="dataset_0.png"
loaded_dataset[1]["image"]="dataset_1.png"
loaded_dataset[2]["image"]="dataset_2.png"

loaded_dataset= loaded_dataset[0:3]

tied_dataset = { 'image':[], 'text':[] }

for i in range(len(loaded_dataset)):
  tied_dataset['image'].append(Image.open(loaded_dataset[i]['image']))
  tied_dataset['text'].append(loaded_dataset[i]['text'])

dataset = Dataset.from_dict(tied_dataset)


dataset_dict = DatasetDict({
    "train": dataset.select(range(2)),  # 첫 2개를 train으로
    "validation": dataset.select(range(2, 3))  # 나머지 1개를 validation으로
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 2
    })
    validation: Dataset({
        features: ['image', 'text'],
        num_rows: 1
    })
})


### Preprocess (tokenize) data

In [ ]:
def preprocess_function(examples):

  #  이미지 전처리, Prompt는 Deplot 기본 프롬프트와 동일
  inputs = processor(
    images=examples["image"],
    text="Generate underlying data table of the figure below:",
    return_tensors="pt",
    max_length=398,
    padding="max_length",
    truncation=True
  )

  # text 전처리 (토큰화)
  labels = processor.tokenizer(
    examples["text"],
    return_tensors="pt",
    max_length=398,
    padding="max_length",
    truncation=True
  )

  return {
    "flattened_patches": inputs["flattened_patches"].squeeze(0),  # 배치 차원 제거
    "attention_mask": inputs["attention_mask"].squeeze(0),
    "labels": labels["input_ids"].squeeze(0)
  }

# 데이터셋 전처리
tokenized_dataset = dataset_dict.map(preprocess_function,batched = True) #실제 데이터셋으로는 이거 실행
#tokenized_dataset = dataset_dict.map(preprocess_function)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Arial.TTF:   0%|          | 0.00/276k [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Train DEPLOT model

In [ ]:
class DeplotTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False,  num_items_in_batch=None):

    labels = inputs.get("labels")
    outputs = model(**inputs)
    logits = outputs.logits

    return (outputs.loss, outputs) if return_outputs else outputs.loss

training_args = TrainingArguments(
  output_dir=project_dir+"/results",  # 모델 출력 중간저장 경로
  evaluation_strategy="epoch",
  per_device_train_batch_size=3,      # 배치 크기 (GPU 부하에 영향)
  per_device_eval_batch_size=12,       # Evaluation Batch size
  gradient_accumulation_steps=2,      # Gradient Accum (GPU 부하 감소)
  num_train_epochs=6,                 # Epoch
  logging_dir="./logs",               # Log 기록 (디스크에 저장)
  logging_steps=10,                   # 로그 기록 간격
  save_strategy="epoch",              # 모델 저장 -> Epoch마다
  learning_rate=5e-5,                 # 학습률
  weight_decay=0.01,                  # Weight Decay
  save_total_limit=2,                 # 저장할 체크포인트 수 최대 2개
  fp16=True,                           # FP16 (GPU 부하 감소)
  report_to="none"
)

# Trainer 인스턴스 생성
trainer = DeplotTrainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset["train"],
  eval_dataset=tokenized_dataset["validation"],
  tokenizer=processor
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-973549465af7>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DeplotTrainer.__init__`. Use `processing_class` instead.
  trainer = DeplotTrainer(


In [ ]:
#모델 훈련 시작
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.227700,1.114086
2,0.248300,0.253524
4,0.185800,0.226882
5,0.176300,0.228371


TrainOutput(global_step=1998, training_loss=0.5803203356635941, metrics={'train_runtime': 14803.6515, 'train_samples_per_second': 0.811, 'train_steps_per_second': 0.135, 'total_flos': 2.6917209007057797e+19, 'train_loss': 0.5803203356635941, 'epoch': 5.991004497751124})

### Save and Load finetuned model

In [ ]:
save_directory = project_dir+"/models"

model.save_pretrained(save_directory)

In [ ]:
# 저장된 모델과 토크나이저 경로
load_directory = project_dir+"/models"

# 모델 불러오기
loaded_model = Pix2StructForConditionalGeneration.from_pretrained(load_directory).to(device)